### Exercises on `__init__`, `self`, and instance initialization

In the following problems you will practice:
- Writing custom `__init__` methods
- Using `self` to initialize instance attributes
- Using positional arguments, keyword-only arguments, `*args` and `**kwargs`
- Adding simple validation inside `__init__`

Each problem is followed by a reference solution.

#### Problem 1 – Basic `Person` initialization

Create a `Person` class with the following behavior:

1. The initializer (`__init__`) should accept two positional arguments: `first_name` and `last_name`.
2. Store them as instance attributes.
3. Add a `full_name` method that returns the full name as a single string: `"First Last"`.

Example usage:

```python
p = Person("Eric", "Idle")
p.first_name   # 'Eric'
p.last_name    # 'Idle'
p.full_name()  # 'Eric Idle'
```

In [1]:
class Person:
    """Simple representation of a person with a first and last name."""

    def __init__(self, first_name, last_name):
        # Use instance attributes to store state on the object
        self.first_name = first_name
        self.last_name = last_name

    def full_name(self):
        """Return the full name as 'First Last'."""
        return f"{self.first_name} {self.last_name}"


# Quick checks
p = Person("Eric", "Idle")
assert p.first_name == "Eric"
assert p.last_name == "Idle"
assert p.full_name() == "Eric Idle"

#### Problem 2 – N-dimensional point using `*args`

Create a `PointND` class to represent a point in n-dimensional space.

Requirements:

1. The initializer should accept any number of positional coordinates using `*coords`.
2. Store the coordinates in an instance attribute named `coords` as a **tuple**.
3. Add a read-only attribute (property) `dimension` that returns the number of coordinates.
4. Add a method `distance_from_origin()` that returns the Euclidean distance from the origin.

Example usage:

```python
p = PointND(3, 4)
p.coords               # (3, 4)
p.dimension            # 2
p.distance_from_origin()  # 5.0
```

In [2]:
import math

class PointND:
    """Representation of an n-dimensional point using a flexible __init__."""

    def __init__(self, *coords):
        # Store coordinates as a tuple to keep them immutable by convention
        self.coords = tuple(coords)

    @property
    def dimension(self):
        """Return the number of dimensions of this point."""
        return len(self.coords)

    def distance_from_origin(self):
        """Compute the Euclidean distance from the origin (0, 0, ..., 0)."""
        return math.sqrt(sum(c ** 2 for c in self.coords))


# Quick checks
p2 = PointND(3, 4)
assert p2.coords == (3, 4)
assert p2.dimension == 2
assert math.isclose(p2.distance_from_origin(), 5.0)

p3 = PointND(1, 2, 2)
assert p3.dimension == 3
assert math.isclose(p3.distance_from_origin(), 3.0)

#### Problem 3 – Circle with keyword-only argument and validation

Example usage:

```python
c = Circle(radius=1.5)
c.radius    # 1.5
c.diameter  # 3.0
c.area      # approx 7.06858
```

In [3]:
class Circle:
    """Circle defined by its radius.

    The radius must be non-negative. Initialization enforces this invariant.
    """

    def __init__(self, *, radius):
        if radius < 0:
            raise ValueError("radius must be non-negative")
        self.radius = radius

    @property
    def diameter(self):
        return 2 * self.radius

    @property
    def area(self):
        return math.pi * (self.radius ** 2)


# Quick checks
c = Circle(radius=1.5)
assert math.isclose(c.diameter, 3.0)
assert math.isclose(c.area, math.pi * 1.5 ** 2)

try:
    Circle(radius=-1)
except ValueError as ex:
    assert "radius" in str(ex).lower()

#### Problem 4 – `BankAccount` with default and keyword-only arguments

In [4]:
class BankAccount:
    """Simple bank account model with basic validation in __init__."""

    def __init__(self, owner, *, balance=0.0, currency="USD"):
        if balance < 0:
            raise ValueError("initial balance cannot be negative")
        self.owner = owner
        self.balance = float(balance)
        self.currency = currency

    def deposit(self, amount):
        if amount <= 0:
            raise ValueError("deposit amount must be positive")
        self.balance += amount
        return self.balance

    def withdraw(self, amount):
        if amount <= 0:
            raise ValueError("withdrawal amount must be positive")
        if amount > self.balance:
            raise ValueError("insufficient funds")
        self.balance -= amount
        return self.balance


# Quick checks
acc = BankAccount("Guido", balance=100.0, currency="EUR")
assert acc.owner == "Guido"
assert acc.currency == "EUR"
assert math.isclose(acc.balance, 100.0)
assert math.isclose(acc.deposit(50.0), 150.0)
assert math.isclose(acc.withdraw(20.0), 130.0)

try:
    BankAccount("Spam", balance=-10)
except ValueError:
    pass
else:
    raise AssertionError("Expected ValueError for negative initial balance")

try:
    acc.withdraw(1_000_000)
except ValueError:
    pass
else:
    raise AssertionError("Expected ValueError for insufficient funds")

#### Problem 5 – Flexible configuration object using `**kwargs`

In [5]:
class Config:
    """Flexible configuration object using **kwargs in __init__."""

    def __init__(self, **options):
        # Store a copy of the options dictionary
        self.options = dict(options)

        # Also expose each option as an attribute on the instance
        for name, value in options.items():
            setattr(self, name, value)


# Quick checks
cfg = Config(theme="dark", debug=True, timeout=30)
assert cfg.options == {"theme": "dark", "debug": True, "timeout": 30}
assert cfg.theme == "dark"
assert cfg.debug is True
assert cfg.timeout == 30

empty_cfg = Config()
assert empty_cfg.options == {}

#### Problem 6 – Line segment with multiple initialization styles

In [7]:
class LineSegment:
    """2D line segment that accepts multiple __init__ calling styles.

    Valid forms:
        LineSegment(x1, y1, x2, y2)
        LineSegment((x1, y1), (x2, y2))
    """

    def __init__(self, *args):
        # Normalize arguments into p1=(x1, y1), p2=(x2, y2)
        if len(args) == 4:
            x1, y1, x2, y2 = args
            self.p1 = (x1, y1)
            self.p2 = (x2, y2)
        elif len(args) == 2:
            p1, p2 = args
            if (len(p1) == 2) and (len(p2) == 2):
                self.p1 = tuple(p1)
                self.p2 = tuple(p2)
            else:
                raise TypeError("Expected two 2-tuples for points p1 and p2")
        else:
            raise TypeError(
                "LineSegment expects either 4 coordinates or 2 points ((x1, y1), (x2, y2))"
            )

    def length(self):
        (x1, y1) = self.p1
        (x2, y2) = self.p2
        return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


# Quick checks
seg1 = LineSegment(0, 0, 3, 4)
seg2 = LineSegment((0, 0), (3, 4))
assert math.isclose(seg1.length(), 5.0)
assert math.isclose(seg2.length(), 5.0)
assert seg1.p1 == (0, 0)
assert seg1.p2 == (3, 4)

try:
    LineSegment(1, 2, 3)  # invalid
except TypeError:
    pass
else:
    raise AssertionError("Expected TypeError for invalid number of arguments")